## Aula 13 - APIs

In [ ]:
import requests

In [ ]:
url = 'https://swapi.co/api/people/'
response = requests.get(url)
response_json = response.json()
#print(response_json)
#print(response_json['name'])

In [ ]:
from tkinter import *
class TkUrlImage():
    def __init__(self, window, url, width = 400, height = 400):
        self.url = url
        self.window = window
        self.width = width
        self.height = height

        raw_data = requests.get(self.url, stream=True).raw.data
        im = Image.open(io.BytesIO(raw_data))
        self.image = ImageTk.PhotoImage(im)
        self.me = Label(self.window, image=self.image, width=self.width, height=self.height)
    
    def as_label(self):
        return self.me
        
    

In [ ]:
import requests
import io
from tkinter import *
from PIL import Image, ImageTk

class Meowner():
    def __init__(self):
        self.janela = Tk()
        self.janela.title('Meowner')
        self.janela.geometry('800x600')
        self.janela.tk_setPalette(background= '#7e7d80', foreground = '#ffFF00', activeBackground = 'blue')
        
        self.btn_update = Button(self.janela, text='OTRO GATO!', command=self.update_cat)
        self.btn_update.pack()
        
        self.update_cat()
        
        self.janela.mainloop()
        
        
    def update_cat(self):
        try:
            self.image.destroy()
        except:
            pass
        
        self.cat_img_url = self.get_random_cat()
        self.image = TkUrlImage(window=self.janela, url=self.cat_img_url, width=600, height=400).as_label()
        self.image.pack()

    def get_random_cat(self):
        url = 'https://aws.random.cat/meow'
        r = requests.get(url)
        r_json = r.json()
        return r_json['file']

m = Meowner()

In [ ]:
import requests
import io
from tkinter import *
from PIL import Image, ImageTk

class Meowner():
    def __init__(self):
        self.janela = Tk()
        self.janela.title('Meowner')
        self.janela.geometry('800x600')
        self.janela.tk_setPalette(background= '#7e7d80', foreground = '#ffFF00', activeBackground = 'blue')
        
        self.btn_update = Button(self.janela, text='OTRO GATO!', command=self.update_cat)
        self.btn_update.pack()
        
        self.update_cat()
        
        self.janela.mainloop()
        
        
    def update_cat(self):
        self.cat_img_url = self.get_random_cat()
        self.image = self.get_tkinter_image(self.cat_img_url)
        try:
            self.label.destroy()
        except:
            pass
        self.label = Label(self.janela, image=self.image, width=600, height=400)
        self.label.pack()
        
    def get_tkinter_image(self, img_url):
        raw_data = requests.get(img_url, stream=True).raw.data
        im = Image.open(io.BytesIO(raw_data))
        image = ImageTk.PhotoImage(im)
        return image

    def get_random_cat(self):
        url = 'https://aws.random.cat/meow'
        r = requests.get(url)
        r_json = r.json()
        return r_json['file']

m = Meowner()

## Gatoflix - Com API Requests

In [12]:
import requests
import csv
import io
from tkinter import *
from tkinter import messagebox
from PIL import Image, ImageTk

class Gatoflix():
    def __init__(self):
        self.base_filmes = self.get_movie_base()
        self.do_layout()

    def get_movie_base(self):
        base = {}
        with open('D:\GOOGLE DRIVE\BAUEN CONTROL\[11] PYTHON - LETS CODE\movies.csv', 'r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                titulo = row['title']
                base[titulo] = row
                
        return base
    
    def do_movie_detail(self, evt):
        widget = evt.widget # isso volta o listbox que foi clicado
        index = widget.curselection()
        titulo_filme = widget.get(index)
        
        omdb_info = self.fetch_omdb_detail(titulo_filme)
        if omdb_info is not None:
            self.update_labels(omdb_info)
    
    def fetch_omdb_detail(self, title):
        
        # Aqui vc faz a requisição para a API do OMDB
        '''
            esse método tem que voltar um dicionário com as seguintes
            chaves:
            title, year, plot, awards e poster
        '''
        title_fix = title[:title.index('(')].strip()
        year = title[title.index('('):title.index(')')].strip()
        url = 'https://www.omdbapi.com/?apikey=c3a8f2a3&t='+title_fix
        response = requests.get(url)   
        response_json = response.json()
        
        if 'Error' in response:
            massagebox.showinfo('ERRO!','Filme não Encontrado no OmDB')
                          
        return {
            'year': response_json ['Year'],
            'plor': response_json ['Plot'],
            'poster': response_json ['Poster'],
            'awards': response['Awards'],
            'title': title
                       
        }
    
        
    def update_labels(self, infos):
        self.label_movie_title['text'] = '{}'.format(infos['title'])
        self.label_year['text'] = 'Ano: {}'.format(infos['year'])
        self.label_awards['text'] = 'Prêmios: {}'.format(infos['awards'])
        self.set_poster(infos['poster'], self.frame_image)
        
    def set_poster(self, url, window):
        self.image = self.get_tkinter_image(url)
        try:
            self.label_img.destroy()
        except:
            pass
        self.label_img = Label(window, image=self.image, width=600, height=400)
        self.label_img.pack()
        
    def get_tkinter_image(self, img_url):
        raw_data = requests.get(img_url, stream=True).raw.data
        im = Image.open(io.BytesIO(raw_data))
        image = ImageTk.PhotoImage(im)
        return image
        
    
    def do_layout(self):
        self.janela = Tk()
        self.janela.title('GATOFLIX')
        self.janela.geometry('1024x768')
        
        lbl_programa = Label(self.janela, text = "Gatoflix", font=('arial', 40, 'bold'))
        lbl_programa.pack()
        
        frame_detalhe = Frame(self.janela)
        frame_detalhe.pack(side = TOP)
        
        self.list_box = Listbox(frame_detalhe, width = 45)
        self.list_box.bind('<<ListboxSelect>>', self.do_movie_detail)
        self.list_box.pack()
        
        self.info = Frame(self.janela)
        self.info.pack(side=LEFT)
        
        self.label_movie_title = Label(self.info, text = "Aqui vai o texto", font=('arial', 32, 'italic'))
        self.label_movie_title.pack(side=TOP)
                
        self.label_year = Label(self.info, text = "Ano: ----", font=('arial', 20, 'bold'))
        self.label_year.pack(side=TOP)
        
        self.label_awards = Label(self.info, text = "Prêmios: ----", font=('arial', 20, 'bold'))
        self.label_awards.pack(side=TOP)
        
        # Inserindo no final do list box o titulo de cada filme
        # da base carregada no início
        for filme in sorted(self.base_filmes.keys()):
            self.list_box.insert(END, filme)
            
        self.frame_image = Frame(self.janela)
        self.frame_image.pack(side=LEFT)
            
        url = 'https://m.media-amazon.com/images/M/MV5BMDU2ZWJlMjktMTRhMy00ZTA5LWEzNDgtYmNmZTEwZTViZWJkXkEyXkFqcGdeQXVyNDQ2OTk4MzI@._V1_SX300.jpg'
        self.set_poster(url, self.frame_image)
        
        self.janela.mainloop()

In [ ]:
gatoflix = Gatoflix()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\rafael britto\appdata\local\programs\python\python37-32\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-12-92b5341c151d>", line 28, in do_movie_detail
    omdb_info = self.fetch_omdb_detail(titulo_filme)
  File "<ipython-input-12-92b5341c151d>", line 53, in fetch_omdb_detail
    'awards': response['Awards'],
TypeError: 'Response' object is not subscriptable
